### Import libraries

In [38]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
# Loading the wallet transaction data
file_path = r"C:\Users\ASUS\OneDrive\Desktop\Greedy Game\Task 3\3_wallet_data.csv"
df = pd.read_csv(file_path)

### Exploratory Data Analysis

In [40]:
# Basic data exploration
print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())

Dataset shape: (1048575, 6)

Column names:
['adv_id', 'transaction_type', 'value', 'currency', 'source_entity_type', 'created_at']


In [41]:
# Display first few rows
df.head()

,adv_id,transaction_type,value,currency,source_entity_type,created_at
0,d016c006-f927-48bc-987d-40a2f3f4648b,DEBIT,1.01,rupiyo_token,OFFER_REWARD,2024-10-21 15:48:09.797749+00:00
1,d016c006-f927-48bc-987d-40a2f3f4648b,DEBIT,0.02,rupiyo_token,DAILY_CHECKIN,2024-10-21 15:48:22.378338+00:00
2,d016c006-f927-48bc-987d-40a2f3f4648b,DEBIT,0.05,rupiyo_token,DAILY_CHECKIN,2024-10-22 02:52:52.695171+00:00
3,d016c006-f927-48bc-987d-40a2f3f4648b,DEBIT,0.10,rupiyo_token,DAILY_CHECKIN,2024-10-23 03:08:36.180386+00:00
4,ed359404-3d8a-4f3e-9d4e-38e6ef575254,DEBIT,1.01,rupiyo_token,OFFER_REWARD,2024-10-21 15:48:09.797749+00:00


In [42]:
# Check data types and basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   adv_id              1047979 non-null  object 
 1   transaction_type    1048575 non-null  object 
 2   value               1048575 non-null  float64
 3   currency            1048575 non-null  object 
 4   source_entity_type  1048575 non-null  object 
 5   created_at          1048575 non-null  object 
dtypes: float64(1), object(5)
memory usage: 48.0+ MB


In [43]:
# Basic statistics
df.describe()

,value
count,1.048575e+06
mean,2.778373e+02
std,1.043008e+03
min,0.000000e+00
25%,3.885000e+01
50%,1.137500e+02
75%,2.600000e+02
max,9.323820e+04


### Data type handling
Since the datetime function has time some places and doesn't have time at some places, we will be using mixed format

In [44]:
# Convert created_at to datetime (handling mixed formats)
df['created_at'] = pd.to_datetime(df['created_at'], format='mixed')

In [45]:
# Check transaction types
print("Transaction types:")
print(df['transaction_type'].value_counts())

Transaction types:
transaction_type
DEBIT    1048575
Name: count, dtype: int64


In [46]:
# Check currencies
print("\nCurrency distribution:")
print(df['currency'].value_counts())


Currency distribution:
currency
offerwall_319     202073
offerwall_59      118763
sikka              62680
rupiyo_token       61403
offerwall_1048     54662
                   ...  
offerwall_630          2
offerwall_410          1
offerwall_697          1
offerwall_356          1
offerwall_380          1
Name: count, Length: 134, dtype: int64


In [47]:
# Check source entity types
print("\nSource entity types:")
print(df['source_entity_type'].value_counts())


Source entity types:
source_entity_type
OFFER_REWARD            906521
DAILY_CHECKIN            69994
SPIN_WHEEL               30827
BITLABS_SURVEY           28518
REFERRAL                  9654
TRANSACTION_REVERSAL      1960
ADJOE                      988
MOCK_OFFER_REWARD           76
CONTEST_REWARD              37
Name: count, dtype: int64


In [48]:
# Sort data by user and date
df = df.sort_values(['adv_id', 'created_at'])

In [49]:
# Create user-level transaction summary
user_stats = df.groupby('adv_id').agg({
    'value': ['count', 'sum', 'mean', 'std', 'min', 'max'],
    'created_at': ['min', 'max'],
    'transaction_type': lambda x: x.value_counts().to_dict(),
    'currency': lambda x: x.nunique(),
    'source_entity_type': lambda x: x.nunique()
}).round(2)

In [50]:
# Flatten column names
user_stats.columns = ['_'.join(col).strip() for col in user_stats.columns]
user_stats = user_stats.reset_index()

In [51]:
# Calculate additional metrics
user_stats['total_transactions'] = user_stats['value_count']
user_stats['total_amount'] = user_stats['value_sum']
user_stats['avg_transaction'] = user_stats['value_mean']
user_stats['amount_std'] = user_stats['value_std']

In [52]:
# Calculate transaction frequency (transactions per day)
user_stats['date_range'] = (user_stats['created_at_max'] - user_stats['created_at_min']).dt.days + 1
user_stats['txn_frequency'] = user_stats['total_transactions'] / user_stats['date_range']

In [53]:
# Extract DEBIT and CREDIT counts from transaction_type column
def extract_txn_counts(txn_dict):
    if pd.isna(txn_dict) or txn_dict == '{}':
        return 0, 0
    txn_dict = eval(txn_dict) if isinstance(txn_dict, str) else txn_dict
    debit_count = txn_dict.get('DEBIT', 0)
    credit_count = txn_dict.get('CREDIT', 0)
    return debit_count, credit_count

user_stats[['debit_count', 'credit_count']] = user_stats['transaction_type_<lambda>'].apply(
    lambda x: pd.Series(extract_txn_counts(x))
)

In [54]:
# Calculate debit/credit ratios
user_stats['debit_ratio'] = user_stats['debit_count'] / user_stats['total_transactions']
user_stats['credit_ratio'] = user_stats['credit_count'] / user_stats['total_transactions']

In [55]:
# Calculate separate amounts for DEBIT and CREDIT
debit_amounts = df[df['transaction_type'] == 'DEBIT'].groupby('adv_id')['value'].sum()
credit_amounts = df[df['transaction_type'] == 'CREDIT'].groupby('adv_id')['value'].sum()
user_stats['total_debit_amount'] = user_stats['adv_id'].map(debit_amounts).fillna(0)
user_stats['total_credit_amount'] = user_stats['adv_id'].map(credit_amounts).fillna(0)

In [56]:
# Calculate net balance and transaction imbalance
user_stats['net_balance'] = user_stats['total_credit_amount'] - user_stats['total_debit_amount']
user_stats['amount_imbalance'] = abs(user_stats['net_balance'])

In [57]:
# Identify statistical outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (data[column] < lower_bound) | (data[column] > upper_bound)

In [58]:
# Apply outlier detection to key metrics
user_stats['outlier_total_amount'] = detect_outliers_iqr(user_stats, 'total_amount')
user_stats['outlier_avg_transaction'] = detect_outliers_iqr(user_stats, 'avg_transaction')
user_stats['outlier_txn_frequency'] = detect_outliers_iqr(user_stats, 'txn_frequency')
user_stats['outlier_amount_imbalance'] = detect_outliers_iqr(user_stats, 'amount_imbalance')

In [59]:
# Create composite anomaly score
user_stats['anomaly_score'] = (
    user_stats['outlier_total_amount'].astype(int) +
    user_stats['outlier_avg_transaction'].astype(int) +
    user_stats['outlier_txn_frequency'].astype(int) +
    user_stats['outlier_amount_imbalance'].astype(int)
)

In [60]:
# Identify high-risk users based on anomaly score
high_risk_users = user_stats[user_stats['anomaly_score'] >= 2].copy()
print(f"Users with high anomaly scores (>=2): {len(high_risk_users)}")

Users with high anomaly scores (>=2): 12228


In [61]:
# Additional fraud indicators
user_stats['very_high_frequency'] = user_stats['txn_frequency'] > user_stats['txn_frequency'].quantile(0.95)
user_stats['very_high_amount'] = user_stats['total_amount'] > user_stats['total_amount'].quantile(0.95)
user_stats['only_debits'] = user_stats['credit_count'] == 0
user_stats['only_credits'] = user_stats['debit_count'] == 0

In [62]:
# Calculate Z-scores for key metrics
from scipy import stats
user_stats['z_score_amount'] = np.abs(stats.zscore(user_stats['total_amount']))
user_stats['z_score_frequency'] = np.abs(stats.zscore(user_stats['txn_frequency']))
user_stats['z_score_avg_txn'] = np.abs(stats.zscore(user_stats['avg_transaction']))

In [63]:
# Identify users with extreme Z-scores
extreme_users = user_stats[
    (user_stats['z_score_amount'] > 3) | 
    (user_stats['z_score_frequency'] > 3) | 
    (user_stats['z_score_avg_txn'] > 3)
].copy()

print(f"Users with extreme Z-scores (>3): {len(extreme_users)}")

Users with extreme Z-scores (>3): 3886


In [64]:
# Time-based anomaly detection
df['hour'] = df['created_at'].dt.hour
df['day_of_week'] = df['created_at'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6])

In [65]:
# Unusual timing patterns
unusual_timing = df.groupby('adv_id').agg({
    'hour': lambda x: len(set(x)),
    'is_weekend': 'mean',
    'created_at': lambda x: x.dt.date.nunique()
}).rename(columns={
    'hour': 'unique_hours',
    'is_weekend': 'weekend_ratio',
    'created_at': 'active_days'
})

In [66]:
# Merge timing data with user stats
user_stats = user_stats.merge(unusual_timing, left_on='adv_id', right_index=True, how='left')

In [67]:
# Identify users with unusual timing patterns
user_stats['unusual_timing'] = (
    (user_stats['weekend_ratio'] > 0.8) |  # Heavy weekend activity
    (user_stats['unique_hours'] <= 2)      # Activity concentrated in very few hours
)

In [68]:
# Create final fraud risk score
user_stats['fraud_risk_score'] = (
    user_stats['anomaly_score'] * 0.3 +
    user_stats['very_high_frequency'].astype(int) * 0.2 +
    user_stats['very_high_amount'].astype(int) * 0.2 +
    (user_stats['only_debits'] | user_stats['only_credits']).astype(int) * 0.2 +
    user_stats['unusual_timing'].astype(int) * 0.1
)

In [69]:
# Classify users by risk level
def classify_risk(score):
    if score >= 0.8:
        return 'Very High Risk'
    elif score >= 0.6:
        return 'High Risk'
    elif score >= 0.4:
        return 'Medium Risk'
    else:
        return 'Low Risk'

user_stats['risk_category'] = user_stats['fraud_risk_score'].apply(classify_risk)

In [70]:
# Summary of risk categories
print("Risk category distribution:")
print(user_stats['risk_category'].value_counts())

Risk category distribution:
risk_category
Low Risk          114661
Very High Risk     12228
High Risk           6466
Medium Risk         3121
Name: count, dtype: int64


In [71]:
# Top suspicious users
top_suspicious = user_stats.nlargest(10, 'fraud_risk_score')[
    ['adv_id', 'fraud_risk_score', 'risk_category', 'total_transactions', 
     'total_amount', 'txn_frequency', 'debit_ratio', 'credit_ratio']
]

print("\nTop 10 most suspicious users:")
print(top_suspicious)


Top 10 most suspicious users:
                                      adv_id  fraud_risk_score  \
556     00fed42c-bb7c-4548-bb4b-39ca52153dda               1.9   
72415   8795a4e5-f808-4f2e-b1bd-2b1564cd7e19               1.9   
103618  c2493a80-27a3-4823-a240-715077afd3a5               1.9   
117651  dcd6adcf-1778-460b-9ba2-1f209cdc6899               1.9   
122207  e5656f49-6f3e-4b31-944a-dea37839a050               1.9   
124474  e99c3065-9237-4eee-8610-02588ee32015               1.9   
40      00111b7f-5eb1-4713-804c-14903398e1ba               1.8   
218     00647267-52a7-4b91-8293-23610a4eac3d               1.8   
382     00aab9f9-0447-4b82-b3a4-d5ecfbf6ff18               1.8   
415     00bcb239-62c3-4f83-a4d1-5e7c25dd8270               1.8   

         risk_category  total_transactions  total_amount  txn_frequency  \
556     Very High Risk                   8       5941.43          8.000   
72415   Very High Risk                   8      19890.00          8.000   
103618  Very High

In [72]:
# Fraud indicators summary
fraud_indicators = {
    'Very High Transaction Frequency': len(user_stats[user_stats['very_high_frequency']]),
    'Very High Transaction Amounts': len(user_stats[user_stats['very_high_amount']]),
    'Only Debit Transactions': len(user_stats[user_stats['only_debits']]),
    'Only Credit Transactions': len(user_stats[user_stats['only_credits']]),
    'Unusual Timing Patterns': len(user_stats[user_stats['unusual_timing']]),
    'High Anomaly Score (>=2)': len(user_stats[user_stats['anomaly_score'] >= 2])
}

print("\nFraud Indicators Summary:")
for indicator, count in fraud_indicators.items():
    print(f"{indicator}: {count} users")


Fraud Indicators Summary:
Very High Transaction Frequency: 6804 users
Very High Transaction Amounts: 6810 users
Only Debit Transactions: 136476 users
Only Credit Transactions: 0 users
Unusual Timing Patterns: 35269 users
High Anomaly Score (>=2): 12228 users


In [73]:
# Detailed analysis of high-risk users
high_risk_detailed = user_stats[user_stats['risk_category'].isin(['High Risk', 'Very High Risk'])].copy()

print(f"\nDetailed analysis of {len(high_risk_detailed)} high-risk users:")
print(f"Average transactions: {high_risk_detailed['total_transactions'].mean():.2f}")
print(f"Average total amount: {high_risk_detailed['total_amount'].mean():.2f}")
print(f"Average frequency: {high_risk_detailed['txn_frequency'].mean():.2f} txns/day")


Detailed analysis of 18694 high-risk users:
Average transactions: 22.17
Average total amount: 10461.92
Average frequency: 5.05 txns/day


In [74]:
# Create final results summary
results_summary = pd.DataFrame({
    'Risk Level': ['Very High Risk', 'High Risk', 'Medium Risk', 'Low Risk'],
    'User Count': [
        len(user_stats[user_stats['risk_category'] == 'Very High Risk']),
        len(user_stats[user_stats['risk_category'] == 'High Risk']),
        len(user_stats[user_stats['risk_category'] == 'Medium Risk']),
        len(user_stats[user_stats['risk_category'] == 'Low Risk'])
    ]
})

results_summary['Percentage'] = (results_summary['User Count'] / len(user_stats) * 100).round(2)

print("\nFraud Risk Summary:")
print(results_summary)


Fraud Risk Summary:
       Risk Level  User Count  Percentage
0  Very High Risk       12228        8.96
1       High Risk        6466        4.74
2     Medium Risk        3121        2.29
3        Low Risk      114661       84.02
